In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_fp = '../input/tabular-playground-series-may-2022/train.csv'
train_df = pd.read_csv(train_fp)

test_fp = '../input/tabular-playground-series-may-2022/test.csv'
test_df = pd.read_csv(test_fp)


In [ ]:
train_df.info()

In [ ]:
train_df.head()

In [ ]:
test_df.info()

In [ ]:
test_df.head()

In [ ]:
train = train_df.copy()
test = test_df.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

train_f_27 = encoder.fit_transform(train['f_27'])
test_f_27 = encoder.fit_transform(test['f_27'])

In [ ]:
train['f_27'] = train_f_27
test['f_27'] = test_f_27

In [ ]:
X = train.drop(['id', 'target'], axis=1)
y = train.target

test = test.drop(['id'], axis=1)

In [ ]:
X

In [ ]:
from sklearn.model_selection import StratifiedKFold, GroupKFold, train_test_split, cross_val_score

xtrain,xvalid,ytrain,yvalid = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
import optuna
from optuna.samplers import TPESampler
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import log_loss, accuracy_score, mean_absolute_error, r2_score, roc_auc_score
from optuna.integration import XGBoostPruningCallback

def objective(trial):
    
    params = {
        "objective": trial.suggest_categorical('objective', ["binary:logistic"]),
        "eval_metric": trial.suggest_categorical('eval_metric', ["auc"]),
        "use_label_encoder": trial.suggest_categorical('use_label_encoder', [False]),
        "n_estimators": trial.suggest_categorical('n_estimators', [40000]),
        "learning_rate": trial.suggest_loguniform('learning_rate', 0.15, 1.0),
        "subsample": trial.suggest_float('subsample', 0.1, 1, step=0.01),
        "colsample_bytree": trial.suggest_float('colsample_bytree', 0.05, 1, step=0.01),
        "max_depth": trial.suggest_int("max_depth", 1, 8),
        "booster": trial.suggest_categorical('booster', ["gbtree"]),
        "gamma": trial.suggest_float('gamma', 0, 100, step=0.1),
        "tree_method": trial.suggest_categorical('tree_method', ["gpu_hist"]),
        "reg_lambda": trial.suggest_loguniform('reg_lambda', 0.1, 100),
        "reg_alpha": trial.suggest_loguniform('reg_alpha', 0.1, 100),
        "random_state": trial.suggest_categorical('random_state', [42]),
        "n_jobs": trial.suggest_categorical('n_jobs', [4]),
        "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
        "gpu_id":0,
            }
    
    #opt_params = params
    #opt_params['n_estimators'] = 80000
    
    model = XGBClassifier(**params)

    model.fit(
        xtrain, 
        ytrain,
        early_stopping_rounds=100,
        eval_set=[(xvalid, yvalid)],
        #eval_metric='auc',
        verbose=False
    )

    preds = model.predict(xvalid)
    #preds = model.predict_proba(xvalid)
    yhat = preds#[:, 1]
    return roc_auc_score(yvalid, yhat)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_params)

In [ ]:
print("Best value (ROC_AUC_SCORE): {:.5f}".format(study.best_value))
print("Best params:")

for key, value in study.best_params.items():
    print("{}: {}".format(key, value))

In [ ]:
xgb_params = study.best_params
xgb_params

In [ ]:
from sklearn.metrics import roc_curve, auc

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

preds = []
scores = []

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    xgb = XGBClassifier(**xgb_params, 
          predictor='gpu_predictor', 
          gpu_id=0)

    xgb.fit(
        xtrain, 
        ytrain,
        #eval_metric='auc',
        early_stopping_rounds=100,
        eval_set=[(xvalid, yvalid)], 
        verbose=False
        )
    
    pred_valid = xgb.predict(X_valid)#[:,1]
    #fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    #score = auc(fpr, tpr)
    score = accuracy_score(y_valid, pred_valid)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('||'*40)
    
    test_preds = xgb.predict(test)#[:,1]
    preds.append(test_preds)
    
print(f"Overall Validation Score: {np.mean(scores)}")

In [ ]:
preds_stack = np.mean(np.column_stack(preds),axis=1).astype(int)
print(preds_stack)

In [ ]:
sample = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv')

submission = sample.copy()

In [ ]:
submission['target'] = preds_stack

submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)